# Regression ANN with best parameters
    find the best approach for learning rate (exponential schedueling - or not)

In [3]:
import pandas as pd;
import numpy as np;
import sklearn
from sklearn.model_selection import train_test_split, cross_validate, cross_val_predict;
from sklearn.preprocessing import MinMaxScaler;
from sklearn import metrics;
from sklearn.model_selection import TimeSeriesSplit;

mae_cv = []
mse_cv = []
mae_gen = []
mse_gen  =[]
rmse_gen = []
mae_nor = []
mae_spi = []
mse_nor = []
mse_spi = []
rmse_nor = []
rmse_spi = []

# Results for lr = 0.001:

In [4]:
count = 0

# data
data = pd.read_csv('Data_set_1_smaller.csv', index_col = 0)

# COMPLETE
data = data.loc[data.index > 2018090000, :]
    
# reset index
data.reset_index(inplace = True)
data.drop('index', axis = 1, inplace = True)
    
# Divide features and labels
X = data.iloc[:, 0:15]
y = data.loc[:, 'Offers']

X.fillna(method = 'ffill', inplace = True)
y.fillna(method = 'ffill', inplace = True)
    
X = X.astype('float64')
X = X.round(20)
    
# divide data into train and test with 20% test data
X_train, X_test, y_train, y_test = train_test_split(
             X, y, test_size=0.2, shuffle=False)
    
# feature scaling
sc_X = MinMaxScaler()
X_train = sc_X.fit_transform(X_train)
X_test = sc_X.transform(X_test)
    
import keras
from keras.models import Sequential # to initialise the NN
from keras.layers import Dense # to create layers
from keras.layers import Dropout
from keras import initializers
from keras import optimizers
from keras.wrappers.scikit_learn import KerasRegressor
    
# possible debug
X_train = np.nan_to_num(X_train)
X_test = np.nan_to_num(X_test)
    
def regressor_tunning(n_hidden = 5, 
                      n_neurons = 40,  
                      kernel_initializer = "he_normal",
                      bias_initializer = initializers.Ones()):
    model = Sequential()
    model.add(Dense(units = n_neurons, input_dim = 15))        
    model.add(keras.layers.LeakyReLU(alpha = 0.2))
    model.add(Dropout(rate = 0.1))        
    for layer in range(n_hidden):
        model.add(Dense(n_neurons))
        model.add(keras.layers.LeakyReLU(alpha = 0.2))
        model.add(Dropout(rate = 0.1))
    model.add(Dense(units = 1, activation = 'linear'))
    optimizer = optimizers.Adamax(lr = 0.001)
    model.compile(loss = 'mse', optimizer = optimizer, metrics = ['mse', 'mae'])
    return model

tscv = TimeSeriesSplit(n_splits = 7)

hist_list = pd.DataFrame()
count = 1
    
regressor = regressor_tunning()
    
for train_index, test_index in tscv.split(X_train):
    X_train_split, X_test_split = X_train[train_index], X_train[test_index]
    y_train_split, y_test_split = y_train[train_index], y_train[test_index]
    hist = regressor.fit(X_train_split, y_train_split, batch_size = 15, epochs = 80)
    hist_list = hist_list.append(hist.history, ignore_index = True)
    print(count)
    count = count + 1

a = []
b = []

for i in range(len(hist_list.mse)):
    a.append(np.mean(hist_list.mse[i]))
    b.append(np.mean(hist_list.mae[i]))

mse_cv.append(np.mean(a))
mae_cv.append(np.mean(b))

# predict for X_test  
y_pred = regressor.predict(X_test)

from sklearn.metrics import mean_squared_error as mse
from sklearn.metrics import mean_absolute_error as mae

rmse_error = mse(y_test, y_pred, squared = False)
mse_error = mse(y_test, y_pred) # 1479.61335
mae_error = mae(y_test, y_pred) # 23.1525

rmse_gen.append(rmse_error)
mse_gen.append(mse_error)
mae_gen.append(mae_error)

# =============================================================================
# Metrics evaluation on spike regions
# =============================================================================

y_spike_occ = pd.read_csv('Spike_binary_1std.csv', usecols = [6])

# create array same size as y_test
y_spike_occ = y_spike_occ.iloc[- len(y_test):]
y_spike_occ = pd.Series(y_spike_occ.iloc[:,0]).values


# smal adjustment
y_test.replace(0, 0.0001,inplace = True)


# select y_pred and y_test only for regions with spikes
y_test_spike = (y_test.T * y_spike_occ).T
y_pred_spike = (y_pred.T * y_spike_occ).T
y_test_spike = y_test_spike[y_test_spike != 0]
y_pred_spike = y_pred_spike[y_pred_spike != 0]

# calculate metric
rmse_spike = mse(y_test_spike, y_pred_spike, squared = False)
mse_spike = mse(y_test_spike, y_pred_spike)
mae_spike = mae(y_test_spike, y_pred_spike)

rmse_spi.append(rmse_spike)
mse_spi.append(mse_spike)
mae_spi.append(mae_spike)

# =============================================================================
# Metric evaluation on normal regions
# =============================================================================

# inverse y_spike_occ so the only normal occurences are chosen
y_normal_occ = (y_spike_occ - 1) * (-1)

# sanity check
y_normal_occ.sum() + y_spike_occ.sum() # gives the correct total 

# select y_pred and y_test only for normal regions
y_test_normal = (y_test.T * y_normal_occ).T
y_pred_normal = (y_pred.T * y_normal_occ).T
y_test_normal = y_test_normal[y_test_normal != 0.00]
y_pred_normal = y_pred_normal[y_pred_normal != 0.00]

# calculate metric
rmse_normal = mse(y_test_normal, y_pred_normal, squared = False)
mse_normal = mse(y_test_normal, y_pred_normal)
mae_normal = mae(y_test_normal, y_pred_normal)

rmse_nor.append(rmse_normal)
mse_nor.append(mse_normal)
mae_nor.append(mae_normal)

C:\Users\maria\anaconda3\lib\site-packages\pandas\core\frame.py:4153: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  downcast=downcast,


Epoch 1/80
40/40 [==============================] - 0s 3ms/step - loss: 17405.4434 - mse: 17405.4434 - mae: 113.1076
Epoch 2/80
40/40 [==============================] - 0s 3ms/step - loss: 14586.1201 - mse: 14586.1201 - mae: 99.5965
Epoch 3/80
40/40 [==============================] - 0s 2ms/step - loss: 6413.8477 - mse: 6413.8477 - mae: 44.7643
Epoch 4/80
40/40 [==============================] - 0s 3ms/step - loss: 4982.5356 - mse: 4982.5356 - mae: 30.2355
Epoch 5/80
40/40 [==============================] - 0s 3ms/step - loss: 5022.3843 - mse: 5022.3843 - mae: 31.3917
Epoch 6/80
40/40 [==============================] - 0s 3ms/step - loss: 5037.6724 - mse: 5037.6724 - mae: 31.4848
Epoch 7/80
40/40 [==============================] - 0s 3ms/step - loss: 5036.8940 - mse: 5036.8936 - mae: 30.6263
Epoch 8/80
40/40 [==============================] - 0s 3ms/step - loss: 4922.5034 - mse: 4922.5034 - mae: 29.2518
Epoch 9/80
40/40 [==============================] - 0s 3ms/step - loss: 4837.8628 -

40/40 [==============================] - 0s 2ms/step - loss: 5066.1079 - mse: 5066.1079 - mae: 29.2030
Epoch 73/80
40/40 [==============================] - 0s 2ms/step - loss: 4937.0806 - mse: 4937.0806 - mae: 29.4870
Epoch 74/80
40/40 [==============================] - 0s 3ms/step - loss: 4795.5947 - mse: 4795.5947 - mae: 29.0222
Epoch 75/80
40/40 [==============================] - 0s 2ms/step - loss: 4908.7134 - mse: 4908.7134 - mae: 28.9514
Epoch 76/80
40/40 [==============================] - 0s 3ms/step - loss: 4819.6855 - mse: 4819.6855 - mae: 29.2689
Epoch 77/80
40/40 [==============================] - 0s 3ms/step - loss: 5033.4722 - mse: 5033.4722 - mae: 29.9331
Epoch 78/80
40/40 [==============================] - 0s 3ms/step - loss: 4855.7129 - mse: 4855.7129 - mae: 28.8137
Epoch 79/80
40/40 [==============================] - 0s 2ms/step - loss: 4819.1343 - mse: 4819.1343 - mae: 28.4469
Epoch 80/80
40/40 [==============================] - 0s 1ms/step - loss: 4653.2676 - mse: 46

79/79 [==============================] - 0s 3ms/step - loss: 3402.1643 - mse: 3402.1643 - mae: 30.5494
Epoch 64/80
79/79 [==============================] - 0s 2ms/step - loss: 3498.1089 - mse: 3498.1089 - mae: 30.6818
Epoch 65/80
79/79 [==============================] - 0s 3ms/step - loss: 3471.5337 - mse: 3471.5337 - mae: 31.0931
Epoch 66/80
79/79 [==============================] - 0s 3ms/step - loss: 3493.7056 - mse: 3493.7056 - mae: 30.3355
Epoch 67/80
79/79 [==============================] - 0s 2ms/step - loss: 3397.8958 - mse: 3397.8958 - mae: 30.0990
Epoch 68/80
79/79 [==============================] - 0s 3ms/step - loss: 3581.5837 - mse: 3581.5837 - mae: 30.5266
Epoch 69/80
79/79 [==============================] - 0s 3ms/step - loss: 3403.4346 - mse: 3403.4346 - mae: 29.5355
Epoch 70/80
79/79 [==============================] - 0s 3ms/step - loss: 3374.0320 - mse: 3374.0320 - mae: 29.3623
Epoch 71/80
79/79 [==============================] - 0s 3ms/step - loss: 3438.7505 - mse: 34

118/118 [==============================] - 0s 2ms/step - loss: 2631.8508 - mse: 2631.8508 - mae: 27.1297
Epoch 53/80
118/118 [==============================] - 0s 2ms/step - loss: 2659.9722 - mse: 2659.9722 - mae: 28.2603
Epoch 54/80
118/118 [==============================] - 0s 1ms/step - loss: 2717.2566 - mse: 2717.2566 - mae: 28.0395
Epoch 55/80
118/118 [==============================] - 0s 2ms/step - loss: 2660.4819 - mse: 2660.4819 - mae: 27.5682
Epoch 56/80
118/118 [==============================] - 0s 3ms/step - loss: 2626.3979 - mse: 2626.3979 - mae: 27.8439
Epoch 57/80
118/118 [==============================] - 0s 3ms/step - loss: 2693.0208 - mse: 2693.0208 - mae: 27.7292
Epoch 58/80
118/118 [==============================] - 0s 3ms/step - loss: 2711.1938 - mse: 2711.1938 - mae: 27.5464
Epoch 59/80
118/118 [==============================] - 0s 3ms/step - loss: 2697.8611 - mse: 2697.8611 - mae: 28.5117
Epoch 60/80
118/118 [==============================] - 0s 3ms/step - loss: 2

157/157 [==============================] - 0s 3ms/step - loss: 2339.7317 - mse: 2339.7317 - mae: 27.4293
Epoch 42/80
157/157 [==============================] - 0s 3ms/step - loss: 2300.5586 - mse: 2300.5586 - mae: 27.3336
Epoch 43/80
157/157 [==============================] - 0s 3ms/step - loss: 2355.5588 - mse: 2355.5588 - mae: 27.5776
Epoch 44/80
157/157 [==============================] - 0s 3ms/step - loss: 2350.0386 - mse: 2350.0386 - mae: 27.5529
Epoch 45/80
157/157 [==============================] - 0s 2ms/step - loss: 2313.3892 - mse: 2313.3892 - mae: 27.3310
Epoch 46/80
157/157 [==============================] - 0s 1ms/step - loss: 2325.9443 - mse: 2325.9441 - mae: 26.9847
Epoch 47/80
157/157 [==============================] - 0s 1ms/step - loss: 2309.2759 - mse: 2309.2759 - mae: 27.0237
Epoch 48/80
157/157 [==============================] - 0s 2ms/step - loss: 2337.3848 - mse: 2337.3848 - mae: 27.2713
Epoch 49/80
157/157 [==============================] - 0s 2ms/step - loss: 2

196/196 [==============================] - 1s 3ms/step - loss: 2046.4398 - mse: 2046.4398 - mae: 26.5859
Epoch 32/80
196/196 [==============================] - 0s 2ms/step - loss: 2119.8613 - mse: 2119.8613 - mae: 26.9062
Epoch 33/80
196/196 [==============================] - 0s 1ms/step - loss: 2096.7937 - mse: 2096.7937 - mae: 26.7589
Epoch 34/80
196/196 [==============================] - 0s 2ms/step - loss: 2101.3965 - mse: 2101.3965 - mae: 26.7960
Epoch 35/80
196/196 [==============================] - 0s 2ms/step - loss: 2046.1245 - mse: 2046.1245 - mae: 26.3448
Epoch 36/80
196/196 [==============================] - 1s 3ms/step - loss: 2099.2759 - mse: 2099.2759 - mae: 26.9357
Epoch 37/80
196/196 [==============================] - 1s 3ms/step - loss: 2096.9099 - mse: 2096.9099 - mae: 26.7734
Epoch 38/80
196/196 [==============================] - 1s 3ms/step - loss: 2055.7825 - mse: 2055.7825 - mae: 26.6418
Epoch 39/80
196/196 [==============================] - 0s 3ms/step - loss: 2

235/235 [==============================] - 1s 3ms/step - loss: 2536.7659 - mse: 2536.7659 - mae: 26.5565
Epoch 21/80
235/235 [==============================] - 0s 2ms/step - loss: 2496.6550 - mse: 2496.6550 - mae: 26.4360
Epoch 22/80
235/235 [==============================] - 0s 1ms/step - loss: 2485.8599 - mse: 2485.8599 - mae: 26.6717
Epoch 23/80
235/235 [==============================] - 1s 3ms/step - loss: 2488.3740 - mse: 2488.3738 - mae: 26.4215
Epoch 24/80
235/235 [==============================] - 1s 3ms/step - loss: 2488.3254 - mse: 2488.3254 - mae: 26.6530
Epoch 25/80
235/235 [==============================] - 1s 2ms/step - loss: 2495.9890 - mse: 2495.9890 - mae: 26.2617: 0s - loss: 2536.6323 - mse: 2536.6323 - mae: 26.30
Epoch 26/80
235/235 [==============================] - 0s 2ms/step - loss: 2467.6394 - mse: 2467.6394 - mae: 26.3668
Epoch 27/80
235/235 [==============================] - 1s 2ms/step - loss: 2420.5325 - mse: 2420.5325 - mae: 26.2335
Epoch 28/80
235/235 [===

274/274 [==============================] - 1s 3ms/step - loss: 2523.7449 - mse: 2523.7449 - mae: 27.3224
Epoch 10/80
274/274 [==============================] - 1s 3ms/step - loss: 2524.8613 - mse: 2524.8613 - mae: 27.0561
Epoch 11/80
274/274 [==============================] - 0s 2ms/step - loss: 2532.2834 - mse: 2532.2832 - mae: 27.1876
Epoch 12/80
274/274 [==============================] - 0s 2ms/step - loss: 2522.0391 - mse: 2522.0391 - mae: 26.9066
Epoch 13/80
274/274 [==============================] - 1s 3ms/step - loss: 2521.4434 - mse: 2521.4434 - mae: 27.0433
Epoch 14/80
274/274 [==============================] - 1s 3ms/step - loss: 2483.7773 - mse: 2483.7773 - mae: 26.7818
Epoch 15/80
274/274 [==============================] - 1s 2ms/step - loss: 2511.4250 - mse: 2511.4250 - mae: 26.9771
Epoch 16/80
274/274 [==============================] - 1s 3ms/step - loss: 2540.4377 - mse: 2540.4377 - mae: 26.8121
Epoch 17/80
274/274 [==============================] - 1s 3ms/step - loss: 2

274/274 [==============================] - 1s 3ms/step - loss: 2405.6899 - mse: 2405.6899 - mae: 26.5196
Epoch 79/80
274/274 [==============================] - 1s 3ms/step - loss: 2444.7437 - mse: 2444.7437 - mae: 26.7356
Epoch 80/80
274/274 [==============================] - 1s 3ms/step - loss: 2431.3303 - mse: 2431.3303 - mae: 26.1820
7


In [5]:
rmse_cv = []
for i in mse_cv:
    rmse_cv.append(i ** 0.5)
    
results = pd.DataFrame({'rmse_cv':rmse_cv,
              
                        'mae_cv': mae_cv,
                        
                        'rmse_general': rmse_gen, 
                 
                        'mae_general': mae_gen,
                        
                        'rmse_spike': rmse_spi,
                 
                        'mae_spike': mae_spi,
                        
                        'rmse_normal': rmse_nor,
                    
                        'mae_normal': mae_nor})

results

,rmse_cv,mae_cv,rmse_general,mae_general,rmse_spike,mae_spike,rmse_normal,mae_normal
0,54.473353,28.298016,35.831947,28.778041,64.724014,52.024334,29.394322,25.418225


# Results for exponential schedueling:

In [6]:
count = 0

# data
data = pd.read_csv('Data_set_1_smaller.csv', index_col = 0)
data = data.loc[data.index > 2018090000, :]
    
# reset index
data.reset_index(inplace = True)
data.drop('index', axis = 1, inplace = True)
    
# Divide features and labels
X = data.iloc[:, 0:15]
y = data.loc[:, 'Offers']

X.fillna(method = 'ffill', inplace = True)
y.fillna(method = 'ffill', inplace = True)
    
X = X.astype('float64')
X = X.round(20)
    
# divide data into train and test with 20% test data
X_train, X_test, y_train, y_test = train_test_split(
             X, y, test_size=0.2, shuffle=False)
    
# feature scaling
sc_X = MinMaxScaler()
X_train = sc_X.fit_transform(X_train)
X_test = sc_X.transform(X_test)
    
import keras
from keras.models import Sequential # to initialise the NN
from keras.layers import Dense # to create layers
from keras.layers import Dropout
from keras import initializers
from keras import optimizers
from keras.wrappers.scikit_learn import KerasRegressor
    
# possible debug
X_train = np.nan_to_num(X_train)
X_test = np.nan_to_num(X_test)
    
def regressor_tunning(n_hidden = 5, 
                      n_neurons = 40,  
                      kernel_initializer = "he_normal",
                      bias_initializer = initializers.Ones()):
    model = Sequential()
    model.add(Dense(units = n_neurons, input_dim = 15))        
    model.add(keras.layers.LeakyReLU(alpha = 0.2))
    model.add(Dropout(rate = 0.1))        
    for layer in range(n_hidden):
        model.add(Dense(n_neurons))
        model.add(keras.layers.LeakyReLU(alpha = 0.2))
        model.add(Dropout(rate = 0.1))
    model.add(Dense(units = 1, activation = 'linear'))
    optimizer = optimizers.Adamax(lr = 0.001)
    model.compile(loss = 'mse', optimizer = optimizer, metrics = ['mse', 'mae'])
    return model

def exponential_decay_fn(epoch):
    return 0.001 * 0.1 ** (epoch / 20)

tscv = TimeSeriesSplit(n_splits = 7)
    
hist_list = pd.DataFrame()
count = 1
    
regressor = regressor_tunning()
    
for train_index, test_index in tscv.split(X_train):
    X_train_split, X_test_split = X_train[train_index], X_train[test_index]
    y_train_split, y_test_split = y_train[train_index], y_train[test_index]
    lr_scheduler = keras.callbacks.LearningRateScheduler(exponential_decay_fn)
    hist = regressor.fit(X_train_split, y_train_split, batch_size = 15, epochs = 80, callbacks = [lr_scheduler])
    hist_list = hist_list.append(hist.history, ignore_index = True)
    print(count)
    count = count + 1

a = []
b = []

for i in range(len(hist_list.mse)):
    a.append(np.mean(hist_list.mse[i]))
    b.append(np.mean(hist_list.mae[i]))

mse_cv.append(np.mean(a))
mae_cv.append(np.mean(b))

# predict for X_test  
y_pred = regressor.predict(X_test)

from sklearn.metrics import mean_squared_error as mse
from sklearn.metrics import mean_absolute_error as mae

rmse_error = mse(y_test, y_pred, squared = False)
mse_error = mse(y_test, y_pred) # 1479.61335
mae_error = mae(y_test, y_pred) # 23.1525

rmse_gen.append(rmse_error)
mse_gen.append(mse_error)
mae_gen.append(mae_error)

# =============================================================================
# Metrics evaluation on spike regions
# =============================================================================

y_spike_occ = pd.read_csv('Spike_binary_1std.csv', usecols = [6])

# create array same size as y_test
y_spike_occ = y_spike_occ.iloc[- len(y_test):]
y_spike_occ = pd.Series(y_spike_occ.iloc[:,0]).values


# smal adjustment
y_test.replace(0, 0.0001,inplace = True)


# select y_pred and y_test only for regions with spikes
y_test_spike = (y_test.T * y_spike_occ).T
y_pred_spike = (y_pred.T * y_spike_occ).T
y_test_spike = y_test_spike[y_test_spike != 0]
y_pred_spike = y_pred_spike[y_pred_spike != 0]

# calculate metric
rmse_spike = mse(y_test_spike, y_pred_spike, squared = False)
mse_spike = mse(y_test_spike, y_pred_spike)
mae_spike = mae(y_test_spike, y_pred_spike)

rmse_spi.append(rmse_spike)
mse_spi.append(mse_spike)
mae_spi.append(mae_spike)

# =============================================================================
# Metric evaluation on normal regions
# =============================================================================

# inverse y_spike_occ so the only normal occurences are chosen
y_normal_occ = (y_spike_occ - 1) * (-1)

# sanity check
y_normal_occ.sum() + y_spike_occ.sum() # gives the correct total 

# select y_pred and y_test only for normal regions
y_test_normal = (y_test.T * y_normal_occ).T
y_pred_normal = (y_pred.T * y_normal_occ).T
y_test_normal = y_test_normal[y_test_normal != 0.00]
y_pred_normal = y_pred_normal[y_pred_normal != 0.00]

# calculate metric
rmse_normal = mse(y_test_normal, y_pred_normal, squared = False)
mse_normal = mse(y_test_normal, y_pred_normal)
mae_normal = mae(y_test_normal, y_pred_normal)

rmse_nor.append(rmse_normal)
mse_nor.append(mse_normal)
mae_nor.append(mae_normal)

C:\Users\maria\anaconda3\lib\site-packages\pandas\core\frame.py:4153: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  downcast=downcast,


Epoch 1/80
40/40 [==============================] - 0s 1ms/step - loss: 17588.0742 - mse: 17588.0742 - mae: 113.9518 - lr: 0.0010
Epoch 2/80
40/40 [==============================] - 0s 1ms/step - loss: 16462.4746 - mse: 16462.4746 - mae: 108.8403 - lr: 8.9125e-04
Epoch 3/80
40/40 [==============================] - 0s 1ms/step - loss: 12250.9766 - mse: 12250.9766 - mae: 86.3100 - lr: 7.9433e-04
Epoch 4/80
40/40 [==============================] - 0s 1ms/step - loss: 5672.8530 - mse: 5672.8530 - mae: 38.8023 - lr: 7.0795e-04
Epoch 5/80
40/40 [==============================] - 0s 1ms/step - loss: 5053.1982 - mse: 5053.1982 - mae: 30.8409 - lr: 6.3096e-04
Epoch 6/80
40/40 [==============================] - 0s 1ms/step - loss: 5008.1108 - mse: 5008.1108 - mae: 30.3591 - lr: 5.6234e-04
Epoch 7/80
40/40 [==============================] - 0s 1ms/step - loss: 5024.3276 - mse: 5024.3276 - mae: 32.2963 - lr: 5.0119e-04
Epoch 8/80
40/40 [==============================] - 0s 1ms/step - loss: 4980.38

40/40 [==============================] - 0s 1ms/step - loss: 4770.6533 - mse: 4770.6533 - mae: 31.0334 - lr: 8.9125e-07
Epoch 63/80
40/40 [==============================] - 0s 1ms/step - loss: 4757.0908 - mse: 4757.0908 - mae: 30.4086 - lr: 7.9433e-07
Epoch 64/80
40/40 [==============================] - 0s 1ms/step - loss: 5133.6655 - mse: 5133.6650 - mae: 31.5788 - lr: 7.0795e-07
Epoch 65/80
40/40 [==============================] - 0s 2ms/step - loss: 5300.8174 - mse: 5300.8174 - mae: 31.5409 - lr: 6.3096e-07
Epoch 66/80
40/40 [==============================] - 0s 3ms/step - loss: 4854.8823 - mse: 4854.8823 - mae: 30.2035 - lr: 5.6234e-07
Epoch 67/80
40/40 [==============================] - 0s 3ms/step - loss: 4788.5088 - mse: 4788.5088 - mae: 31.2958 - lr: 5.0119e-07
Epoch 68/80
40/40 [==============================] - 0s 2ms/step - loss: 5320.6777 - mse: 5320.6777 - mae: 31.1921 - lr: 4.4668e-07
Epoch 69/80
40/40 [==============================] - 0s 3ms/step - loss: 5044.6763 - mse

79/79 [==============================] - 0s 2ms/step - loss: 3863.9558 - mse: 3863.9558 - mae: 33.8383 - lr: 7.0795e-06
Epoch 45/80
79/79 [==============================] - 0s 2ms/step - loss: 3702.9941 - mse: 3702.9941 - mae: 33.2800 - lr: 6.3096e-06
Epoch 46/80
79/79 [==============================] - 0s 3ms/step - loss: 3729.9456 - mse: 3729.9460 - mae: 33.3767 - lr: 5.6234e-06
Epoch 47/80
79/79 [==============================] - 0s 2ms/step - loss: 3749.6755 - mse: 3749.6755 - mae: 32.6990 - lr: 5.0119e-06
Epoch 48/80
79/79 [==============================] - 0s 1ms/step - loss: 3737.8320 - mse: 3737.8320 - mae: 33.0973 - lr: 4.4668e-06
Epoch 49/80
79/79 [==============================] - 0s 1ms/step - loss: 3914.8486 - mse: 3914.8486 - mae: 34.1018 - lr: 3.9811e-06
Epoch 50/80
79/79 [==============================] - 0s 1ms/step - loss: 3847.5056 - mse: 3847.5056 - mae: 33.1975 - lr: 3.5481e-06
Epoch 51/80
79/79 [==============================] - 0s 1ms/step - loss: 3692.7891 - mse

Epoch 26/80
118/118 [==============================] - 0s 2ms/step - loss: 3008.1104 - mse: 3008.1104 - mae: 31.2001 - lr: 5.6234e-05
Epoch 27/80
118/118 [==============================] - 0s 3ms/step - loss: 2976.7329 - mse: 2976.7329 - mae: 30.8058 - lr: 5.0119e-05
Epoch 28/80
118/118 [==============================] - 0s 2ms/step - loss: 2986.9739 - mse: 2986.9739 - mae: 30.1889 - lr: 4.4668e-05
Epoch 29/80
118/118 [==============================] - 0s 2ms/step - loss: 2941.0928 - mse: 2941.0928 - mae: 31.1195 - lr: 3.9811e-05
Epoch 30/80
118/118 [==============================] - 0s 3ms/step - loss: 2991.2573 - mse: 2991.2573 - mae: 30.7329 - lr: 3.5481e-05
Epoch 31/80
118/118 [==============================] - 0s 3ms/step - loss: 2944.5000 - mse: 2944.5000 - mae: 30.3743 - lr: 3.1623e-05
Epoch 32/80
118/118 [==============================] - 0s 3ms/step - loss: 2935.9297 - mse: 2935.9297 - mae: 30.8512 - lr: 2.8184e-05
Epoch 33/80
118/118 [==============================] - 0s 3ms/

...

157/157 [==============================] - 0s 2ms/step - loss: 2591.6885 - mse: 2591.6885 - mae: 29.1567 - lr: 5.0119e-07
Epoch 68/80
157/157 [==============================] - 0s 1ms/step - loss: 2540.3298 - mse: 2540.3301 - mae: 29.8245 - lr: 4.4668e-07
Epoch 69/80
157/157 [==============================] - 0s 1ms/step - loss: 2597.7021 - mse: 2597.7021 - mae: 29.7849 - lr: 3.9811e-07
Epoch 70/80
157/157 [==============================] - 0s 1ms/step - loss: 2554.6675 - mse: 2554.6675 - mae: 29.8094 - lr: 3.5481e-07
Epoch 71/80
157/157 [==============================] - 0s 1ms/step - loss: 2583.9614 - mse: 2583.9614 - mae: 29.9605 - lr: 3.1623e-07
Epoch 72/80
157/157 [==============================] - 0s 2ms/step - loss: 2559.2490 - mse: 2559.2490 - mae: 30.1129 - lr: 2.8184e-07
Epoch 73/80
157/157 [==============================] - 0s 2ms/step - loss: 2651.8018 - mse: 2651.8018 - mae: 30.3562 - lr: 2.5119e-07
Epoch 74/80
157/157 [==============================] - 0s 1ms/step - loss:

196/196 [==============================] - 0s 2ms/step - loss: 2312.7522 - mse: 2312.7522 - mae: 29.2473 - lr: 4.4668e-06
Epoch 49/80
196/196 [==============================] - 0s 2ms/step - loss: 2357.9368 - mse: 2357.9368 - mae: 29.2980 - lr: 3.9811e-06
Epoch 50/80
196/196 [==============================] - 0s 1ms/step - loss: 2364.9727 - mse: 2364.9727 - mae: 29.5203 - lr: 3.5481e-06
Epoch 51/80
196/196 [==============================] - 0s 1ms/step - loss: 2264.9778 - mse: 2264.9778 - mae: 28.8685 - lr: 3.1623e-06
Epoch 52/80
196/196 [==============================] - 0s 2ms/step - loss: 2340.4482 - mse: 2340.4482 - mae: 29.2745 - lr: 2.8184e-06
Epoch 53/80
196/196 [==============================] - 0s 2ms/step - loss: 2370.4429 - mse: 2370.4429 - mae: 29.4668 - lr: 2.5119e-06
Epoch 54/80
196/196 [==============================] - 0s 1ms/step - loss: 2386.2869 - mse: 2386.2869 - mae: 29.3295 - lr: 2.2387e-06
Epoch 55/80
196/196 [==============================] - 0s 2ms/step - loss:

235/235 [==============================] - 1s 3ms/step - loss: 2689.2661 - mse: 2689.2661 - mae: 28.5412 - lr: 3.9811e-05
Epoch 30/80
235/235 [==============================] - 1s 2ms/step - loss: 2704.5959 - mse: 2704.5959 - mae: 29.0531 - lr: 3.5481e-05
Epoch 31/80
235/235 [==============================] - 1s 2ms/step - loss: 2749.9448 - mse: 2749.9448 - mae: 28.8125 - lr: 3.1623e-05
Epoch 32/80
235/235 [==============================] - 1s 3ms/step - loss: 2680.7791 - mse: 2680.7791 - mae: 28.3525 - lr: 2.8184e-05
Epoch 33/80
235/235 [==============================] - 0s 2ms/step - loss: 2709.2368 - mse: 2709.2368 - mae: 28.6411 - lr: 2.5119e-05
Epoch 34/80
235/235 [==============================] - 0s 2ms/step - loss: 2737.5901 - mse: 2737.5901 - mae: 28.8612 - lr: 2.2387e-05
Epoch 35/80
235/235 [==============================] - 0s 1ms/step - loss: 2666.1511 - mse: 2666.1511 - mae: 28.4255 - lr: 1.9953e-05
Epoch 36/80
235/235 [==============================] - 0s 2ms/step - loss:

274/274 [==============================] - 1s 2ms/step - loss: 2773.0349 - mse: 2773.0349 - mae: 29.1053 - lr: 3.5481e-04
Epoch 11/80
274/274 [==============================] - 1s 3ms/step - loss: 2794.9612 - mse: 2794.9612 - mae: 29.7378 - lr: 3.1623e-04
Epoch 12/80
274/274 [==============================] - 1s 3ms/step - loss: 2841.8315 - mse: 2841.8315 - mae: 29.4881 - lr: 2.8184e-04
Epoch 13/80
274/274 [==============================] - 1s 3ms/step - loss: 2767.3677 - mse: 2767.3677 - mae: 28.8570 - lr: 2.5119e-04
Epoch 14/80
274/274 [==============================] - 1s 3ms/step - loss: 2791.4321 - mse: 2791.4321 - mae: 29.4482 - lr: 2.2387e-04
Epoch 15/80
274/274 [==============================] - 1s 2ms/step - loss: 2756.9170 - mse: 2756.9170 - mae: 29.3514 - lr: 1.9953e-04
Epoch 16/80
274/274 [==============================] - 1s 2ms/step - loss: 2804.3992 - mse: 2804.3992 - mae: 29.4088 - lr: 1.7783e-04
Epoch 17/80
274/274 [==============================] - 0s 2ms/step - loss:

274/274 [==============================] - 0s 1ms/step - loss: 2730.1543 - mse: 2730.1543 - mae: 29.3747 - lr: 3.1623e-07
Epoch 72/80
274/274 [==============================] - 0s 2ms/step - loss: 2775.4521 - mse: 2775.4524 - mae: 29.5240 - lr: 2.8184e-07
Epoch 73/80
274/274 [==============================] - 1s 3ms/step - loss: 2788.0532 - mse: 2788.0532 - mae: 29.3310 - lr: 2.5119e-07
Epoch 74/80
274/274 [==============================] - 1s 3ms/step - loss: 2776.0076 - mse: 2776.0076 - mae: 29.1548 - lr: 2.2387e-07
Epoch 75/80
274/274 [==============================] - 1s 3ms/step - loss: 2762.9089 - mse: 2762.9089 - mae: 29.2446 - lr: 1.9953e-07
Epoch 76/80
274/274 [==============================] - 1s 2ms/step - loss: 2783.1274 - mse: 2783.1274 - mae: 29.2790 - lr: 1.7783e-07
Epoch 77/80
274/274 [==============================] - 1s 3ms/step - loss: 2771.1619 - mse: 2771.1619 - mae: 29.1455 - lr: 1.5849e-07
Epoch 78/80
274/274 [==============================] - 1s 3ms/step - loss:

In [7]:
rmse_cv = []
for i in mse_cv:
    rmse_cv.append(i ** 0.5)
    
results = pd.DataFrame({'rmse_cv':rmse_cv,
              
                        'mae_cv': mae_cv,
                        
                        'rmse_general': rmse_gen, 
                 
                        'mae_general': mae_gen,
                        
                        'rmse_spike': rmse_spi,
                 
                        'mae_spike': mae_spi,
                        
                        'rmse_normal': rmse_nor,
                    
                        'mae_normal': mae_nor}, index = ['lr = 0.001', 'exp schedueling'])

results

,rmse_cv,mae_cv,rmse_general,mae_general,rmse_spike,mae_spike,rmse_normal,mae_normal
lr = 0.001,54.473353,28.298016,35.831947,28.778041,64.724014,52.024334,29.394322,25.418225
exp schedueling,56.681972,30.662315,33.294198,25.621725,71.838469,58.977570,22.865370,20.800763


In [8]:
def highlight_min(s):
    '''
    highlight the maximum in a Series yellow.
    '''
    is_max = s == s.min()
    return ['background-color: yellow' if v else '' for v in is_max]

results.style.apply(highlight_min)

,rmse_cv,mae_cv,rmse_general,mae_general,rmse_spike,mae_spike,rmse_normal,mae_normal
lr = 0.001,54.473353,28.298016,35.831947,28.778041,64.724014,52.024334,29.394322,25.418225
exp schedueling,56.681972,30.662315,33.294198,25.621725,71.838469,58.977570,22.865370,20.800763


In [9]:
results.to_csv('Results_tuning_10_exponential_lr_vs_constant_lr_5_layers_40_neurons.csv')